Load jupyter sql extension


In [5]:
#Load Jupter SQL extension
%load_ext sql
%config SqlMagic.autocommit = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Next load the database

In [6]:
#Load database
%sql sqlite:///../data/raw/frantamarit.db

'Connected: @../data/raw/frantamarit.db'

Loading the database tables

In [7]:
#Load database tables
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [8]:
#Showing tables
tables

name
rank_info
personal_info
business_info


1. Creating engine to connect to the database

In [20]:
#from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

2. Create engine and get data tables

In [24]:
#Creating engine
eng = create_engine('sqlite:///../data/raw/frantamarit.db')

3. Creating data frames for each table

In [49]:
#Creating function to ingest sql table and convert it to dataframe
def create_df_sql(x):
    return pd.read_sql_table(x, con=eng)

#Defining dataframes
df_business_info = create_df_sql('business_info')
df_personal_info = create_df_sql('personal_info')
df_rank_info = create_df_sql('rank_info')

In [50]:
#Quick view of rank_info
df_rank_info.head()

,id,name,position,Unnamed: 0
0,5542,aNANT ASAVABHOKHIN & FAMILY,NaN,0
1,4183,ARASH FERDOWSi,NaN,1
2,4761,banthoon LAMSAM & FAmily,NaN,2
3,1449,bilL ADDERLEY & FAMILY,NaN,3
4,1624,BORIS ZINGAREVICH,NaN,4


In [51]:
#Get unique values of column Unnamed: 0
df_rank_info['Unnamed: 0'].unique()

array([   0,    1,    2, ..., 2257, 2258, 2259])

In [52]:
#Looks like the column is redundant, so we will drop it
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in(df_rank_info, 'Unnamed: 0')

In [62]:
#Feature name is not written correctly, so let's harmonized it and lower case
#the name.
def lowercase_feature(x,y):
    x[y] = x[y].apply(lambda z: z.lower())
    return x[y]

lowercase_feature(df_rank_info, 'name')

0       anant asavabhokhin & family
1                    arash ferdowsi
2          banthoon lamsam & family
3            bill adderley & family
4                 boris zingarevich
                   ...             
2255                 zhao xiaoqiang
2256                zhou liangzhang
2257                   zhu xingming
2258                       zhuo jun
2259                    olgun zorlu
Name: name, Length: 2260, dtype: object

In [63]:
#We don want to get rid of the NaN yet, so we'll fill them with 0s.
def fill_0(x):
    return x.fillna(0, inplace=True)

rank_info


,id,name,position
0,5542,anant asavabhokhin & family,NaN
1,4183,arash ferdowsi,NaN
2,4761,banthoon lamsam & family,NaN
3,1449,bill adderley & family,NaN
4,1624,boris zingarevich,NaN


In [40]:
def get_null(x):
    return x.isnull().sum()

get_null(df_personal_info)

id             0
lastName       0
age           65
Unnamed: 0     0
gender        13
country        0
image          0
dtype: int64

In [32]:
len(df_rank_info)
#df_rank_info['Unnamed: 0'].le

2260

In [38]:
print(rank_info.notnull().sum())
print(personal_info.notnull().sum())

NameError: name 'rank_info' is not defined

In [ ]:
#Formula to drop column inplace
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in()

"""
rank_info.drop('Unnamed: 0', axis=1, inplace=True)
personal_info.drop('Unnamed: 0', axis=1, inplace=True)
business_info.drop('Unnamed: 0', axis=1, inplace=True)
"""

In [ ]:
personal_info.head()

In [ ]:
partial_info = pd.merge(rank_info, personal_info, on='id', how='inner')
master_info = pd.merge(partial_info, business_info, on='id', how='inner')
master_info.head()


In [ ]:
#Cleaning and lowercasing name and lastName features
master_info[['name','lastName']] = \
master_info[['name','lastName']].apply(lambda x: x.astype(str).str.lower())

In [ ]:
master_info.isnull().sum()

In [ ]:
#Renaming feature names
master_info.rename(index={e: e+1 for e in range(len(master_info))}, 
                columns={'name': 'full_name', 
                         'lastName': 'last_name',
                        'Source': 'source'}, 
                inplace=True)

In [ ]:
master_info.head()